In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm


pd.options.display.max_rows = 35 
pd.options.display.max_columns = None

In [ ]:
col_ignore = ['scheme_name', 'wpt_name', 'source_class', 'quality_group', 'quantity_group', 'recorded_by', 
             'extraction_type_group', 'extraction_type_class','management_group', 'payment_type', 'region_code',
              'district_code', 'subvillage', 'scheme_management', 'source_type', 'waterpoint_type_group', 
              'num_private', 'funder']

water_values = pd.read_csv('../data/raw/WaterValuesOriginal.csv')
print('Original Shape', water_values.shape)
water_values = water_values[[i for i in water_values.columns if i not in col_ignore]]
water_labels = pd.read_csv('../data/raw/WaterLabelsOriginal.csv')
water_labels['target'] = water_labels.status_group.values  #change target column
water_labels.drop('status_group', axis = 1, inplace = True) # drop the original column

water_values = pd.merge(water_values, water_labels, on = 'id') #merge target and features
water_values['date_recorded_date'] = pd.to_datetime(water_values.date_recorded)
water_values['year_recorded'] = pd.DatetimeIndex(water_values.date_recorded_date).year
#target distribution is roughly the same as the population distribution so we decided to drop them because there
#are not many entried and because the recorded was earlier than construction (all in 2004)

water_values = water_values[water_values.year_recorded >= 2011].reset_index(drop = True)
water_values = water_values[(water_values.source != 'unknown') & (water_values.water_quality != 'unknown') & 
                           (water_values.quantity != 'unknown')]
print('New Shape', water_values.shape)

- After loading the original csv, we decided to drop redundant columns:
`['scheme_name', 'wpt_name', 'source_class', 'quality_group', 'quantity_group', 'recorded_by','extraction_type_group',   'extraction_type_class','management_group', 'payment_type', 'region_code', 'district_code', 'subvillage', 'scheme_management', 'source_type', 'waterpoint_type_group',  'num_private', 'funder']`
- The original dataset had two seperate csv for the features and targets, so we merged them together to create a single dataframe for cleaning
- We also changed any columns that include date to a datetime object
- Finally, we dropped any rows that were from 2011 or later, or had a value unknown in the columns `[source, quantity, water_quality]`
- After merging and dropping rows, the dataframe went from the shape `(59400,40)` to `(57427, 25)`

In [ ]:
def get_time_since_built(cons, rec): 
    '''This function will take arrays that include the construction year and data recorded to return a new array that
    includes the years passed since it was constructed.  This function will take the difference between the year it was constructed
    and the year it was recorded
    
    Input: 
    -------
    cons (array)
    rec (array)
    
    Output: 
    ---------
    time_array (array) = includes the number of years that has passed since construction date'''
    
    time_array = []
    for c, r in zip(cons, rec): 
        r = int(r.split('-')[0])
        c = int(c)
        diff = r -c
        time_array.append(diff)
    return time_array
    
    
def fix_dates(df): 
    '''This function will fix dates by replacing those that are unknown to the median value
    
    Input: 
    -----------
    Dataframe 
    
    Output: 
    ---------
    Dataframe'''
    

    #fixing dates 
    median_con = df[df.construction_year != 0].construction_year.median()
    df.construction_year.replace(0, median_con, inplace = True)
    cons_year = df.construction_year.values 
    rec_date = df.date_recorded.values 
    

    t_array = get_time_since_built(cons_year, rec_date)
    df['time_passed'] = t_array
    df.drop(['construction_year', 'date_recorded_date', 'year_recorded', 'date_recorded'], 
                      axis = 1, inplace = True)
    return df


water_values = fix_dates(water_values)
water_values.head(2)

- `fix_dates` and `get_time_since_built`
- made a new column for the number of years that have passed since construction.  
- Removed the recorded date and construction date
- for cells where construction date was 0, replaced it with the median construction date of the df

In [ ]:
def fix_booleans(df): 
    '''This function will take a dataframe and change the column public_meeting so that it is binary instead of a boolean
    
    Input: 
    --------
    df (pandas dataframe)
    
    Output:
    ---------
    df (pandas dataframe) = where the column public meeting is binary instead of boolean'''

    df.public_meeting.fillna(False, inplace = True)
    df.public_meeting = df.public_meeting.astype(int)
    df.permit.fillna(False, inplace = True)
    df.permit = water_values.permit.astype(int)
    return df
    
water_values = fix_booleans(water_values)
water_values.head(2)

- `fix_booleans`
- changes binary true-false to 0-1
- chanegd the column `public_meeting` and `permit` to binary instead of boolean

In [5]:
def fix_extract_other(x): 
    ''' This function will take an element from a dataframe and determine whether it is part of the group "other".
    If there is no "other", then it will leave it alone and return the same value'''
    if 'other' in x: 
        return 'extract_other'
    else: 
        return x

water_values['extraction_type'] = water_values.extraction_type.map(fix_extract_other)
water_values.tail(2)

,id,amount_tsh,gps_height,installer,longitude,latitude,basin,region,lga,ward,population,public_meeting,permit,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,target,time_passed
59367,31282,0.0,0,Musa,35.861315,-6.378573,Rufiji,Dodoma,Chamwino,Mvumi Makulu,0,1,1,nira/tanira,vwc,never pay,soft,insufficient,shallow well,hand pump,functional,11
59368,26348,0.0,191,World,38.104048,-6.747464,Wami / Ruvu,Morogoro,Morogoro Rural,Ngerengere,150,1,1,nira/tanira,vwc,pay when scheme fails,salty,enough,shallow well,hand pump,functional,9


- `fix_extract_other`
- Combining 'other' in extract 
- before this step, there were multiple values found in the column that included the word "other".  
- To fix this, we included every value with the word other into a single group other

In [6]:
#fixing population to replace value with mdeian population within its basin 
def fix_population_basin(df):
    '''This function will take a dataframe and fix the population column.  To fix it, it will find any rows with the value 
    of 0 and replace it with the median population of that basin'''
    new_df = df.reset_index(drop = True)
    pbar = tqdm(range(len(new_df)), desc = 'Fixing Invalid Population Entries')
    for idx in pbar: 
        pop = new_df.iloc[idx].population 
        if pop != 0: 
            continue 
        basin_value = new_df.iloc[idx].basin
        basin_median_pop = new_df[(new_df.basin == basin_value) & (new_df.population > 0)].population.median() 
        new_df.loc[idx, 'population'] = basin_median_pop
        
    return new_df     

water_values = fix_population_basin(water_values)
water_values.head(2)

Fixing Invalid Population Entries: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 57247/57247 [02:06<00:00, 453.31it/s]


,id,amount_tsh,gps_height,installer,longitude,latitude,basin,region,lga,ward,population,public_meeting,permit,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,target,time_passed
0,69572,6000.0,1390,Roman,34.938093,-9.856322,Lake Nyasa,Iringa,Ludewa,Mundindi,109.0,1,0,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,functional,12
1,8776,0.0,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,Mara,Serengeti,Natta,280.0,0,1,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,functional,3


- `fix_population_basin`
- In this function, missing inputs within the population column were replaced with the median population of that region.  
- Median population was chosen instead of mean because it will avoid incorporating outliers that have massive or very small populations

In [7]:
# find median latitude and longitude by region excluding the bad 

def median_long_lat(df): 
    '''This function will find values that have invalid lat and long and replace it with the median value within its region'''


    median_lat_longs = df.loc[df.longitude != 0].groupby('region').median()[['latitude', 'longitude']]
    median_lat_longs

    min_long = df.longitude.min()
    # the same entries have a bad latitude value as well
    bad_lat = df.loc[df.region == 'Mwanza'].latitude.mode()[0]
    # checking that this is true: same 1593 entries all have long of zero
    df.loc[df.latitude == bad_lat].longitude.value_counts()

    # dict of group medians by region for lat and long
    lat_dict = dict(median_lat_longs.latitude)
    long_dict = dict(median_lat_longs.longitude)
    long_dict

    # make temp df to hold the correct values in the 1593 locations where incorrect
    temp = df.loc[df.longitude == min_long]
    temp['longitude'], temp['latitude'] = temp.region, temp.region
    temp['longitude'] = temp.longitude.map(lambda x: long_dict[x])
    temp['latitude'] = temp.latitude.map(lambda x: lat_dict[x])


    # replace longitudes with group median by region
    df.loc[df['longitude'] == min_long, 'longitude'] = temp['longitude']
    df.iloc[temp.index]

    # replace latitudes with group median by region
    df.loc[df['latitude'] == bad_lat, 'latitude'] = temp['latitude']
    df.iloc[temp.index]
    
    return df

water_values = median_long_lat(water_values)
water_values.head(2)

c:\users\heeeb\anaconda3\envs\codeacademy\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\heeeb\anaconda3\envs\codeacademy\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\heeeb\anaconda3\envs\codeacademy\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,id,amount_tsh,gps_height,installer,longitude,latitude,basin,region,lga,ward,population,public_meeting,permit,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,target,time_passed
0,69572,6000.0,1390,Roman,34.938093,-9.856322,Lake Nyasa,Iringa,Ludewa,Mundindi,109.0,1,0,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,functional,12
1,8776,0.0,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,Mara,Serengeti,Natta,280.0,0,1,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,functional,3


- `median_long_lat` 
- this function will inpute every missing value within long lat with the median value found within its region

In [8]:
#aggregating values that have abandoned to their corresponding type

def fix_quality(x): 
    '''This function will take a row value x and return its aggregate type (if any)'''
    if 'abandoned' in x: 
        new_x = x.split(' ')[0]
        return new_x
    else: 
        return x

water_values.water_quality = water_values.water_quality.map(fix_quality)
water_values.head(2)

,id,amount_tsh,gps_height,installer,longitude,latitude,basin,region,lga,ward,population,public_meeting,permit,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,target,time_passed
0,69572,6000.0,1390,Roman,34.938093,-9.856322,Lake Nyasa,Iringa,Ludewa,Mundindi,109.0,1,0,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,functional,12
1,8776,0.0,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,Mara,Serengeti,Natta,280.0,0,1,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,functional,3


- `fix_quality` 
- this function will create an aggregate group that combines every value within the quality column that has abandoned in it 

In [9]:
water_values.to_csv('../data/processed/WaterUpdated.csv', index = False)

#save cleaned dataframe to csv